In [ ]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Experimental.Agents, *-*"

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Experimental.Agents;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [ ]:
#!import ./Utils/Settings.cs

In [ ]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.IO;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;

In [ ]:
#!import ./plugins/CustomPlugin/GenCLIPlugin.cs
#!import ./plugins/CustomPlugin/RunPlugin.cs

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;

In [ ]:
#pragma warning disable SKEXP0101

List<IAgent> s_agents = new();

In [ ]:
var cli_plugin = KernelPluginFactory.CreateFromType<GenCLIPlugin>();
var run_plugin = KernelPluginFactory.CreateFromType<RunPlugin>();

In [ ]:
#pragma warning disable SKEXP0101

var cliAgent = await new AgentBuilder()
                        .WithOpenAIChatCompletion("gpt-4-turbo-preview", Settings.OpenAIKey)
                        .WithInstructions("You are a cliAgent . You can generate .NET CLI from request. and pass generate .NET CLI to runPlugin")
                        .WithName("cliAgent")
                        .WithDescription("cliAgent")
                        .WithPlugin(cli_plugin)
                        .BuildAsync();

In [ ]:
#pragma warning disable SKEXP0101

var runAgent = await new AgentBuilder()
                        .WithOpenAIChatCompletion("gpt-4-turbo-preview", Settings.OpenAIKey)
                        .WithInstructions("You are a runAgent. Execute the .NET CLI script generated from cliAgent")
                        .WithName("runAgent")
                        .WithDescription("runAgent")
                        .WithPlugin(run_plugin)
                        .BuildAsync();

In [ ]:
#pragma warning disable SKEXP0101

var dotNETAgent = await new AgentBuilder()
                        .WithOpenAIChatCompletion("gpt-4-turbo-preview", Settings.OpenAIKey)
                        .WithInstructions("You are a dotNETAgent . let cliAgent to generation .NET CLI script from request , Then use runAgent to execute the .NET CLI script generated from cliAgent")
                        .WithPlugin(cliAgent.AsPlugin())
                        .WithPlugin(runAgent.AsPlugin())
                        .BuildAsync();

In [ ]:
var messages = new string[]
            {
                "setup a new project - HelloDemoApp",
                "compile this project - HelloDemoApp",
                "run this project - HelloDemoApp",
            };

In [ ]:
#pragma warning disable SKEXP0101

var thread = await dotNETAgent.NewThreadAsync();

In [ ]:
#pragma warning disable SKEXP0101

foreach (var response in messages.Select(m => thread.InvokeAsync(dotNETAgent, m)))
{
    await foreach (var message in response)
    {
        Console.WriteLine($"[{message.Id}]");
        Console.WriteLine($"# {message.Role}: {message.Content}");
    }
}